In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import Legend, LegendItem
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.models import FactorRange

from bokeh.palettes import Category10
from bokeh.transform import dodge

# Pruebas gemma

## Calidad de generación del texto: BLEU

In [2]:
import evaluate

/home/raulmartinez/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def calcular_bleu(df):
    predictions = df["output"]
    references = df["input"]
    bleu = evaluate.load ("bleu")
    results = bleu.compute (predictions = predictions, references = references)
    print (results)
    df = pd.DataFrame({
    "bleu": [results["bleu"]],
    "precisions": [np.mean(results["precisions"])], 
    "brevity_penalty": [results["brevity_penalty"]],
    "length_ratio": [results["length_ratio"]],
    "translation_length": [results["translation_length"]],
    "reference_length": [results["reference_length"]]
    })
    return df

In [4]:
df_gemma_sin_rol_1 = pd.read_csv("./analysis/gemma_sin_rol_1.csv")
df_gemma_con_rol_1 = pd.read_csv("./analysis/gemma_con_rol_1.csv")
df_gemma_sin_rol_0_5 = pd.read_csv("./analysis/gemma_sin_rol_0_5.csv")
df_gemma_con_rol_0_5 = pd.read_csv("./analysis/gemma_con_rol_0_5.csv")
df_gemma_sin_rol_0_1 = pd.read_csv("./analysis/gemma_sin_rol_0_1.csv")
df_gemma_con_rol_0_1 = pd.read_csv("./analysis/gemma_con_rol_0_1.csv")

print("Gemma sin rol 1")
df_bleu_gemma_sin_rol_1 = calcular_bleu(df_gemma_sin_rol_1)
print("Gemma con rol 1")
df_bleu_gemma_con_rol_1 = calcular_bleu(df_gemma_con_rol_1)
print("Gemma sin rol 0.5")
df_bleu_gemma_sin_rol_0_5 = calcular_bleu(df_gemma_sin_rol_0_5)
print("Gemma con rol 0.5")
df_bleu_gemma_con_rol_0_5 = calcular_bleu(df_gemma_con_rol_0_5)
print("Gemma sin rol 0.1")
df_bleu_gemma_sin_rol_0_1 = calcular_bleu(df_gemma_sin_rol_0_1)
print("Gemma con rol 0.1")
df_bleu_gemma_con_rol_0_1 = calcular_bleu(df_gemma_con_rol_0_1)

Gemma sin rol 1
{'bleu': 2.5562811827773874e-07, 'precisions': [0.7607627572116554, 0.26030283726172393, 0.10263118921854174, 0.05803948873526633], 'brevity_penalty': 1.3793543596593262e-06, 'length_ratio': 0.06899456622062017, 'translation_length': 20557, 'reference_length': 297951}
Gemma con rol 1
{'bleu': 1.567980944699892e-07, 'precisions': [0.7672171555336456, 0.2557238639185498, 0.0955716787590693, 0.05142425006301991], 'brevity_penalty': 8.898034684187848e-07, 'length_ratio': 0.06696907570460316, 'translation_length': 20285, 'reference_length': 302901}
Gemma sin rol 0.5
{'bleu': 2.631053367949994e-07, 'precisions': [0.7497828814049986, 0.24305015552099535, 0.09125624204445315, 0.04961530873939633], 'brevity_penalty': 1.5523949872243844e-06, 'length_ratio': 0.06956177358021956, 'translation_length': 20726, 'reference_length': 297951}
Gemma con rol 0.5
{'bleu': 1.3917434414003416e-07, 'precisions': [0.7550899093475999, 0.2517342915606129, 0.09076280987579827, 0.04635962912296702],

In [80]:
def crear_grafico_bleus(dataframes_dict, modelo='gemma', titulo=None):
    # Extraer datos de los DataFrames
    datos_extraidos = []
    
    for nombre_df, df in dataframes_dict.items():
        # Verificar que el DataFrame no esté vacío y tenga la columna 'bleu'
        if df is not None and not df.empty and 'bleu' in df.columns:
            # Extraer información del nombre del DataFrame
            partes = nombre_df.split('_')
            
            # Verificar que el formato sea correcto y sea del modelo deseado
            if len(partes) >= 6 and partes[2] == modelo:  # df_bleu_modelo_rol_temperatura
                rol = f"{partes[3]}_{partes[4]}"  # con_rol o sin_rol
                if len(partes) == 6:  # Caso: temperatura sin punto decimal (ej: _1)
                    temperatura = float(partes[5])
                elif len(partes) == 7:  # Caso: temperatura con punto decimal (ej: _0_5)
                    temperatura = float(f"{partes[5]}.{partes[6]}")
                else:
                    continue
                
                # Obtener valor BLEU (tomar el primer valor si hay múltiples filas)
                valor_bleu = df['bleu'].iloc[0] if len(df) > 0 else 0
                
                datos_extraidos.append({
                    'temperatura': temperatura,
                    'rol': rol,
                    'bleu': valor_bleu,
                    'dataframe': nombre_df
                })
    
    if not datos_extraidos:
        print(f"No se encontraron datos para el modelo '{modelo}'")
        return None
    
    # Convertir a DataFrame para facilitar manipulación
    df_datos = pd.DataFrame(datos_extraidos)
    
    # Obtener temperaturas y roles únicos
    temperaturas = sorted(df_datos['temperatura'].unique())
    roles = sorted(df_datos['rol'].unique())
    
    # Título automático si no se proporciona
    if titulo is None:
        titulo = f"Valores BLEU - {modelo.title()} por Temperatura"
    
    # Crear figura
    p = figure(
        x_range=[str(t) for t in temperaturas],
        title=titulo,
        x_axis_label="Temperatura",
        y_axis_label="Valor BLEU",
        width=800,
        height=500,
        toolbar_location=None
    )
    
    # Colores para los roles
    colores = {
        'con_rol': Category10[10][0],  # Azul
        'sin_rol': Category10[10][1]   # Naranja
    }
    
    # Crear barras para cada rol
    for rol in roles:
        datos_rol = df_datos[df_datos['rol'] == rol].copy()
        
        # Preparar datos para cada temperatura
        temperaturas_rol = []
        valores_bleu = []
        
        for temp in temperaturas:
            temp_data = datos_rol[datos_rol['temperatura'] == temp]
            if not temp_data.empty:
                valores_bleu.append(temp_data['bleu'].iloc[0])
                temperaturas_rol.append(str(temp))
            else:
                valores_bleu.append(0)  # Valor por defecto si no existe
                temperaturas_rol.append(str(temp))
        
        # Crear source para Bokeh
        source = ColumnDataSource(data=dict(
            x=temperaturas_rol,
            y=valores_bleu,
            temperatura=[str(t) for t in temperaturas],
            rol=[rol] * len(temperaturas),
            bleu_valor=valores_bleu
        ))
        
        # Calcular desplazamiento para agrupar barras
        desplazamiento = -0.2 if rol == 'con_rol' else 0.2
        
        # Crear barras
        barras = p.vbar(
            x=dodge('x', desplazamiento, range=p.x_range),
            top='y',
            source=source,
            width=0.35,
            color=colores[rol],
            legend_label=rol.replace('_', ' ').title(),
            alpha=0.8,
            line_color='white',
            line_width=1
        )
        
        # Añadir hover tool específico para estas barras
        hover = HoverTool(
            renderers=[barras],
            tooltips=[
                ("Temperatura", "@temperatura"),
                ("Rol", "@rol"),
                ("BLEU", "@bleu_valor{0.0000}")
            ]
        )
        p.add_tools(hover)
    
    # Configurar leyenda y apariencia
    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    
    # Mejorar etiquetas del eje X
    p.xaxis.major_label_text_font_size = "12px"
    
    return p

In [14]:
dataframes = {
    'df_bleu_gemma_sin_rol_1': df_bleu_gemma_sin_rol_1,
    'df_bleu_gemma_con_rol_1': df_bleu_gemma_con_rol_1,
    'df_bleu_gemma_sin_rol_0_5': df_bleu_gemma_sin_rol_0_5,
    'df_bleu_gemma_con_rol_0_5': df_bleu_gemma_con_rol_0_5,
    'df_bleu_gemma_sin_rol_0_1': df_bleu_gemma_sin_rol_0_1,
    'df_bleu_gemma_con_rol_0_1': df_bleu_gemma_con_rol_0_1
}
# Crear gráfico
grafico_bleu = crear_grafico_bleus(dataframes, modelo='gemma', titulo="Valores BLEU de Gemma por Temperatura y Rol")
# Mostrar gráfico
output_notebook()
show(grafico_bleu)

Loading BokehJS ...

## Comparación entre pruebas con rol y sin rol a diferentes temperaturas

In [19]:
from evaluate import load

### Bert por defecto

In [20]:
# Modelo Bert por defecto
bertscore = load("bertscore")

import numpy as np

In [21]:
def calcular_bertscore_defecto(df_sin_rol, df_con_rol):
    predictions = df_sin_rol["output"]
    references = df_con_rol["output"]
    results = bertscore.compute(predictions=predictions, references=references, lang="en", model_type="distilbert-base-uncased")
    print("Precision:", sum(results["precision"]) / len(results["precision"]))
    print("Recall:", sum(results["recall"]) / len(results["recall"]))
    print("F1:", sum(results["f1"]) / len(results["f1"]))
    df = pd.DataFrame({
    "precision": [np.mean(results["precision"])],
    "recall": [np.mean(results["recall"])],
    "f1": [np.mean(results["f1"])]
    })
    return df

In [22]:
print ("BertScore por defecto a temperatura 1")
df_bertscore_gemma_1 = calcular_bertscore_defecto(df_gemma_sin_rol_1, df_gemma_con_rol_1)
print ("BertScore por defecto a temperatura 0.5")
df_bertscore_gemma_0_5 = calcular_bertscore_defecto(df_gemma_sin_rol_0_5, df_gemma_con_rol_0_5)
print ("BertScore por defecto a temperatura 0.1")
df_bertscore_gemma_0_1 = calcular_bertscore_defecto(df_gemma_sin_rol_0_1, df_gemma_con_rol_0_1)

BertScore por defecto a temperatura 1
Precision: 0.9005568917592367
Recall: 0.9007785137494405
F1: 0.9005949890613556
BertScore por defecto a temperatura 0.5
Precision: 0.8772079618771871
Recall: 0.8757964762051901
F1: 0.876428058942159
BertScore por defecto a temperatura 0.1
Precision: 0.8983016260464987
Recall: 0.8984694226582846
F1: 0.8983149087429046


In [93]:
def crear_grafico_bertscore(dataframes_dict, metrica='f1', titulo=None):
    # Extraer datos de los DataFrames
    datos_extraidos = []

    for nombre_df, df in dataframes_dict.items():
        # Verificar que el DataFrame no esté vacío y tenga la métrica solicitada
        if df is not None and not df.empty and metrica in df.columns:
            # Extraer información del nombre del DataFrame
            partes = nombre_df.split('_')

            # Verificar formato: df_bertscore_modelo_temperatura
            if len(partes) >= 4 and partes[1].startswith('bertscore'):
                modelo = partes[2]

                # Manejar temperatura (con o sin punto decimal)
                if len(partes) == 4: # Caso: temperatura sin punto decimal (ej: _1)
                    temperatura = float(partes[3])
                elif len(partes) == 5:  # Caso: temperatura con punto decimal (ej: _0_5)
                    temperatura = float(f"{partes[3]}.{partes[4]}")
                else:
                    continue

                # Obtener valor de la métrica
                valor_metrica = df[metrica].iloc[0]
                datos_extraidos.append({
                    'temperatura': str(temperatura),
                    'modelo': modelo,
                    'valor': valor_metrica
                })

    if not datos_extraidos:
        print(f"No se encontraron datos para la métrica '{metrica}'")
        return None

    # Convertir a DataFrame para facilitar manipulación
    df_datos = pd.DataFrame(datos_extraidos)
    # Obtener temperaturas y modelos únicos
    temperaturas = sorted(df_datos['temperatura'].unique(), key=lambda x: float(x))
    modelos = sorted(df_datos['modelo'].unique())

    # Crear factores jerárquicos para eje X
    factores_x = [(temp, modelo) for temp in temperaturas for modelo in modelos]
    df_datos['x'] = list(zip(df_datos['temperatura'], df_datos['modelo']))

    # Asignar colores por modelo
    colores_disponibles = Category10[10]
    colores_modelos = {modelo: colores_disponibles[i % len(colores_disponibles)] for i, modelo in enumerate(modelos)}

    # Título automático si no se proporciona
    if titulo is None:
        titulo = f"BERTScore - {metrica.upper()} por Modelo y Temperatura"

    # Crear figura
    p = figure(
        x_range=FactorRange(*factores_x),
        title=titulo,
        x_axis_label="Temperatura y Modelo",
        y_axis_label=metrica.upper(),
        width=max(800, len(factores_x) * 60),
        height=500,
        toolbar_location=None
    )

    legend_items = []
    barras_por_modelo = {}

    for modelo in modelos:
        df_modelo = df_datos[df_datos['modelo'] == modelo]
        source_modelo = ColumnDataSource(data=dict(
            x=df_modelo['x'],
            y=df_modelo['valor'],
            modelo=df_modelo['modelo'],
            temperatura=df_modelo['temperatura'],
            color=[colores_modelos[modelo]] * len(df_modelo),
            metrica_valor=df_modelo['valor']
        ))

        barra = p.vbar(
            x='x',
            top='y',
            source=source_modelo,
            width=0.8,
            line_color='white',
            fill_color='color',
            alpha=0.8
        )

        barras_por_modelo[modelo] = barra  # <- Guardamos para la leyenda

        hover = HoverTool(
            renderers=[barra],
            tooltips=[
                ("Modelo", "@modelo"),
                ("Temperatura", "@temperatura"),
                (f"{metrica.upper()}", "@metrica_valor{0.0000}")
            ]
        )
        p.add_tools(hover)

    # Añadir leyenda usando las barras correctas
    for modelo in modelos:
        legend_items.append(LegendItem(label=modelo.title(), renderers=[barras_por_modelo[modelo]]))

    p.add_layout(Legend(items=legend_items, location="top_right"), 'right')
    p.legend.click_policy = "hide"

    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.xaxis.major_label_orientation = 1
    p.xaxis.major_label_text_font_size = "11px"

    return p

In [77]:
dataframes_bert_def_gemma = {
    'df_bertscore_gemma_1': df_bertscore_gemma_1,
    'df_bertscore_gemma_0_5': df_bertscore_gemma_0_5,
    'df_bertscore_gemma_0_1': df_bertscore_gemma_0_1
}
# Crear gráfico de BERTScore por defecto
grafico_bertscore_defecto = crear_grafico_bertscore(dataframes_bert_def_gemma, metrica='f1', titulo="BERTScore F1 de Gemma por Temperatura")
# Mostrar gráfico
output_notebook()
show(grafico_bertscore_defecto)

Loading BokehJS ...

### Bert utilizando modelo Bio_ClinicalBERT

In [33]:
from bert_score import score

In [34]:
def calcular_bertscore_bioclinical(df_sin_rol, df_con_rol):
    predictions = df_sin_rol["output"].astype(str).tolist()
    references = df_con_rol["output"].astype(str).tolist()

    P, R, F1 = score(predictions, references, lang="en", num_layers=12, idf=False, model_type="emilyalsentzer/Bio_ClinicalBERT")
    print(f"Precission: {sum(P) / len(P)}")
    print(f"Recall: {sum(R) / len(R)}")
    print(f"F1: {sum(F1) / len(F1)}")

    df = pd.DataFrame({
        "precision": [(sum(P) / len(P)).item()],
        "recall": [(sum(R) / len(R)).item()],
        "f1": [(sum(F1) / len(F1)).item()]
    })
    return df

    

In [35]:
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 1")
df_bertscorebioclinical_gemma_1 = calcular_bertscore_bioclinical(df_gemma_sin_rol_1, df_gemma_con_rol_1)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.5")
df_bertscorebioclinical_gemma_0_5 = calcular_bertscore_bioclinical(df_gemma_sin_rol_0_5, df_gemma_con_rol_0_5)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.1")
df_bertscorebioclinical_gemma_0_1 = calcular_bertscore_bioclinical(df_gemma_sin_rol_0_1, df_gemma_con_rol_0_1)

BertScore con modelo Bio_ClinicalBERT a temperatura 1
Precission: 0.872083306312561
Recall: 0.8703686594963074
F1: 0.8710890412330627
BertScore con modelo Bio_ClinicalBERT a temperatura 0.5
Precission: 0.838509738445282
Recall: 0.837006688117981
F1: 0.8376386165618896
BertScore con modelo Bio_ClinicalBERT a temperatura 0.1
Precission: 0.8701820969581604
Recall: 0.8672230839729309
F1: 0.8686009645462036


In [74]:
df_bertscorebioclinical_gemma_1

,precision,recall,f1
0,0.872083,0.870369,0.871089


In [83]:
dataframes_bert_bioclinical_gemma = {
    'df_bertscorebioclinical_gemma_1': df_bertscorebioclinical_gemma_1,
    'df_bertscorebioclinical_gemma_0_5': df_bertscorebioclinical_gemma_0_5,
    'df_bertscorebioclinical_gemma_0_1': df_bertscorebioclinical_gemma_0_1
}
# Crear gráfico de BERTScore con Bio_ClinicalBERT
grafico_bertscore_bioclinical = crear_grafico_bertscore(dataframes_bert_bioclinical_gemma, metrica='f1', titulo="BERTScore F1 de Gemma con Bio_ClinicalBERT por Temperatura")
# Mostrar gráfico
output_notebook()
show(grafico_bertscore_bioclinical)

Loading BokehJS ...

## Medidas de readabilidad y comprensión del texto

In [ ]:
# pip install https://github.com/andreasvc/readability/tarball/master

In [5]:
import re
import nltk
from nltk.probability import FreqDist
from textblob import TextBlob
import readability
import spacy

In [6]:
def evaluate_prompt(prompt_list,max_rows=5):
    # Correctness (basic grammar check with spaCy)

    flesh_scores = []
    smog_scores = []
    dale_scores = []
    sentiment_polarity_scores = []
    sentiment_subjectivity_scores = []
    lexical_diversity_scores = []
    lexical_richness_scores = []

    for prompt_element in prompt_list[:max_rows]:
        prompt = prompt_element['text']

        # Clarity (readability score using Readability)
        results_r = readability.getmeasures(prompt, lang='en')
        flesch_reading_ease = results_r['readability grades']['FleschReadingEase']
        flesh_scores.append(flesch_reading_ease)

        flesch_reading_ease = results_r['readability grades']['FleschReadingEase']
        smog_index = results_r['readability grades']['SMOGIndex']
        smog_scores.append(smog_index)


        dale_chall_score = results_r['readability grades']['DaleChallIndex']
        dale_scores.append(dale_chall_score)

        # Engagement (sentiment analysis with TextBlob)
        blob = TextBlob(prompt)
        sentiment_polarity_scores.append(blob.sentiment.polarity)
        sentiment_subjectivity_scores.append(blob.sentiment.subjectivity)

        words = prompt.split()
        unique_words = set(words)
        lexical_diversity = len(unique_words) / len(words) if words else 0
        lexical_diversity_scores.append(lexical_diversity)

        # Lexical richness (frequency distribution of words)
        fdist = FreqDist(words)
        lexical_richness = sum(fdist.freq(word) for word in unique_words) / len(unique_words) if unique_words else 0
        lexical_richness_scores.append(lexical_richness)

    return flesh_scores, smog_scores, dale_scores, sentiment_polarity_scores, sentiment_subjectivity_scores,lexical_diversity_scores,lexical_richness_scores

  #'], engagement_scores["sentiment_polarity"], engagement_scores["sentiment_subjectivity"], engagement_scores["lexical_diversity"], engagement_scores["lexical_richness"]

In [7]:
def calcular_metricas_comprension(df):
    prompt_list = [{'text': output} for output in df['output']]
    flesh_scores, smog_scores, dale_scores, sentiment_polarity_scores, sentiment_subjectivity_scores, lexical_diversity_scores, lexical_richness_scores = evaluate_prompt(prompt_list)

    print("Flesch Reading Ease Scores:", sum(flesh_scores) / len(flesh_scores))
    print("SMOG Index Scores:", sum(smog_scores) / len(smog_scores))
    print("Dale-Chall Index Scores:", sum(dale_scores) / len(dale_scores))
    print("Sentiment Polarity Scores:", sum(sentiment_polarity_scores) / len(sentiment_polarity_scores))
    print("Sentiment Subjectivity Scores:", sum(sentiment_subjectivity_scores) / len(sentiment_subjectivity_scores))
    print("Lexical Diversity Scores:", sum(lexical_diversity_scores) / len(lexical_diversity_scores))
    print("Lexical Richness Scores:", sum(lexical_richness_scores) / len(lexical_richness_scores))

    df = pd.DataFrame({
        "flesch_reading_ease": [np.mean(flesh_scores)],
        "smog_index": [np.mean(smog_scores)],
        "dale_chall_index": [np.mean(dale_scores)],
        "sentiment_polarity": [np.mean(sentiment_polarity_scores)],
        "sentiment_subjectivity": [np.mean(sentiment_subjectivity_scores)],
        "lexical_diversity": [np.mean(lexical_diversity_scores)],
        "lexical_richness": [np.mean(lexical_richness_scores)]
    })
    return df

In [8]:
print("Metricas de readabilty y comprensión para sin rol a 1")
df_legibilidad_gemma_sin_rol_1 = calcular_metricas_comprension(df_gemma_sin_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 1")
df_legibilidad_gemma_con_rol_1 = calcular_metricas_comprension(df_gemma_con_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.5")
df_legibilidad_gemma_sin_rol_0_5 = calcular_metricas_comprension(df_gemma_sin_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.5")
df_legibilidad_gemma_con_rol_0_5 = calcular_metricas_comprension(df_gemma_con_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.1")
df_legibilidad_gemma_sin_rol_0_1 = calcular_metricas_comprension(df_gemma_sin_rol_0_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.1")
df_legibilidad_gemma_con_rol_0_1 = calcular_metricas_comprension(df_gemma_con_rol_0_1)

Metricas de readabilty y comprensión para sin rol a 1
Flesch Reading Ease Scores: 34.22234708982448
SMOG Index Scores: 13.058184468534472
Dale-Chall Index Scores: 7.898114446416173
Sentiment Polarity Scores: 0.022286375661375663
Sentiment Subjectivity Scores: 0.3416752645502646
Lexical Diversity Scores: 0.7564128854920904
Lexical Richness Scores: 0.014400490012946335
----------------------------------------------------
Metricas de readabilty y comprensión para con rol a 1
Flesch Reading Ease Scores: 31.6450007348268
SMOG Index Scores: 15.185644137160375
Dale-Chall Index Scores: 8.598488592455155
Sentiment Polarity Scores: 0.0637936507936508
Sentiment Subjectivity Scores: 0.5060899470899471
Lexical Diversity Scores: 0.7376177487147453
Lexical Richness Scores: 0.014315781683464557
----------------------------------------------------
Metricas de readabilty y comprensión para sin rol a 0.5
Flesch Reading Ease Scores: 35.348670973085504
SMOG Index Scores: 13.076315167439983
Dale-Chall Index

In [ ]:
def crear_grafico_legibilidad(dataframes_dict, metrica='flesch_reading_ease', titulo=None):
    # Extraer datos de los DataFrames
    datos_extraidos = []
    
    for nombre_df, df in dataframes_dict.items():
        # Verificar que el DataFrame no esté vacío y tenga la métrica solicitada
        if df is not None and not df.empty and metrica in df.columns:
            # Extraer información del nombre del DataFrame
            partes = nombre_df.split('_')
            
            # Verificar formato: df_legibilidad_modelo_rol_temperatura
            # Ejemplo: df_legibilidad_gemma_con_rol_1 o df_legibilidad_gemma_sin_rol_0_5
            if len(partes) >= 6 and partes[1] == 'legibilidad':
                modelo = partes[2]
                rol = f"{partes[3]}_{partes[4]}"
                
                if len(partes) == 6:
                    temperatura = float(partes[5])
                elif len(partes) == 7:
                    temperatura = float(f"{partes[5]}.{partes[6]}")
                else:
                    continue
                
                valor_metrica = df[metrica].iloc[0] if len(df) > 0 else 0
                
                datos_extraidos.append({
                    'temperatura': temperatura,
                    'modelo': modelo,
                    'rol': rol,
                    'valor': valor_metrica,
                    'dataframe': nombre_df
                })

    if not datos_extraidos:
        print(f"No se encontraron datos para la métrica '{metrica}'")
        return None
    
    df_datos = pd.DataFrame(datos_extraidos)
    temperaturas = sorted(df_datos['temperatura'].unique())
    modelos = sorted(df_datos['modelo'].unique())
    roles = sorted(df_datos['rol'].unique())

    # Título automático si no se proporciona
    if titulo is None:
        titulo_formateado = metrica.replace('_', ' ').title()
        titulo = f"Legibilidad - {titulo_formateado} por Modelo, Temperatura y Rol"

    # Crear tuplas jerárquicas para eje X
    etiquetas_x = []
    datos_graficos = []

    for temp in temperaturas:
        for modelo in modelos:
            for rol in roles:
                datos_combinacion = df_datos[
                    (df_datos['temperatura'] == temp) & 
                    (df_datos['modelo'] == modelo) &
                    (df_datos['rol'] == rol)
                ]
                
                etiqueta = (str(temp), modelo, rol)
                etiquetas_x.append(etiqueta)
                
                if not datos_combinacion.empty:
                    valor = datos_combinacion['valor'].iloc[0]
                else:
                    valor = 0
                
                datos_graficos.append({
                    'x': etiqueta,
                    'modelo': modelo,
                    'temperatura': temp,
                    'rol': rol,
                    'valor': valor
                })

    # Crear figura con eje X jerárquico
    p = figure(
        x_range=FactorRange(*etiquetas_x),
        title=titulo,
        x_axis_label="Temperatura - Modelo - Rol",
        y_axis_label=metrica.replace('_', ' ').title(),
        width=max(1000, len(etiquetas_x) * 40),
        height=500,
        toolbar_location=None
    )

    # Colores base para los modelos
    colores_base = Category10[10]
    colores_modelos = {}

    for i, modelo in enumerate(modelos):
        color_base = colores_base[i % len(colores_base)]

        if color_base.startswith('#'):
            r = int(color_base[1:3], 16)
            g = int(color_base[3:5], 16)
            b = int(color_base[5:7], 16)
            colores_modelos[f"{modelo}_con_rol"] = f"rgb({r},{g},{b})"
            colores_modelos[f"{modelo}_sin_rol"] = f"rgba({r},{g},{b},0.5)"
        else:
            colores_modelos[f"{modelo}_con_rol"] = color_base
            colores_modelos[f"{modelo}_sin_rol"] = color_base

    # Crear renderizadores separados por modelo + rol para leyenda correcta
    leyenda_items = []
    for modelo in modelos:
        for rol in roles:
            datos_filtrados = [d for d in datos_graficos if d['modelo'] == modelo and d['rol'] == rol]
            if not datos_filtrados:
                continue
            
            x_vals = [d['x'] for d in datos_filtrados]
            y_vals = [d['valor'] for d in datos_filtrados]
            color = colores_modelos.get(f"{modelo}_{rol}", "gray")
            temperaturas_lista = [d['temperatura'] for d in datos_filtrados]
            modelos_lista = [modelo] * len(x_vals)
            roles_lista = [rol] * len(x_vals)
            
            source = ColumnDataSource(data=dict(
                x=x_vals,
                y=y_vals,
                colores=[color] * len(x_vals),
                modelo=modelos_lista,
                temperatura=temperaturas_lista,
                rol=roles_lista,
                metrica_valor=y_vals
            ))

            barra = p.vbar(
                x='x',
                top='y',
                source=source,
                width=0.8,
                color='colores',
                line_color='white',
                line_width=1
            )

            # Añadir a la leyenda
            rol_texto = "Con Rol" if rol == "con_rol" else "Sin Rol"
            etiqueta_leyenda = f"{modelo.title()} - {rol_texto}"
            leyenda_items.append(LegendItem(label=etiqueta_leyenda, renderers=[barra]))

            # Añadir hover tool solo una vez por grupo
            hover = HoverTool(
                renderers=[barra],
                tooltips=[
                    ("Modelo", "@modelo"),
                    ("Temperatura", "@temperatura"),
                    ("Rol", "@rol"),
                    (metrica.replace('_', ' ').title(), "@metrica_valor{0.0000}")
                ]
            )
            p.add_tools(hover)

    # Añadir leyenda
    legend = Legend(items=leyenda_items, location="top_right")
    p.add_layout(legend, 'right')
    p.legend.click_policy = "hide"

    # Configurar apariencia
    p.xgrid.grid_line_color = None
    # Determinar el valor mínimo de Y para ajustar el rango automáticamente
    # Cambiado p.y_range.start = 0 a las siguientes dos lineas
    min_y = min([d['valor'] for d in datos_graficos])
    p.y_range.start = min_y if min_y < 0 else 0
    p.xaxis.major_label_text_font_size = "9px"
    p.xaxis.major_label_orientation = 1

    return p

In [10]:
dataframes_legilidad_gemma = {
    'df_legibilidad_gemma_sin_rol_1': df_legibilidad_gemma_sin_rol_1,
    'df_legibilidad_gemma_con_rol_1': df_legibilidad_gemma_con_rol_1,
    'df_legibilidad_gemma_sin_rol_0_5': df_legibilidad_gemma_sin_rol_0_5,
    'df_legibilidad_gemma_con_rol_0_5': df_legibilidad_gemma_con_rol_0_5,
    'df_legibilidad_gemma_sin_rol_0_1': df_legibilidad_gemma_sin_rol_0_1,
    'df_legibilidad_gemma_con_rol_0_1': df_legibilidad_gemma_con_rol_0_1
}
# Crear gráfico de legibilidad
grafico_legibilidad = crear_grafico_legibilidad(dataframes_legilidad_gemma, metrica='flesch_reading_ease', titulo="Legibilidad de Gemma por Temperatura y Rol")
# Mostrar gráfico
output_notebook()
show(grafico_legibilidad)

Loading BokehJS ...

## Calcular longitud media de los textos

In [12]:
# Calcular longitud media de los textos por palabras
def calculate_average_word_count(text_list):
    total_words = sum(len(text.split()) for text in text_list)
    average_word_count = total_words / len(text_list)
    return average_word_count

In [13]:
def calcular_longitud_texto_y_respuesta(df):
    longitud_textos = calculate_average_word_count(df['input'])
    longitud_respuestas = calculate_average_word_count(df['output'])

    print(f"Longitud de los textos: {longitud_textos}")
    print(f"Longitud de las respuestas: {longitud_respuestas}")

    df = pd.DataFrame({
        "longitud_textos": [longitud_textos],
        "longitud_respuestas": [longitud_respuestas]
    })
    return df

In [43]:
print("Longitud textos y respuestas sin rol a 1")
calcular_longitud_texto_y_respuesta(df_gemma_sin_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 1")
calcular_longitud_texto_y_respuesta(df_gemma_con_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.5")
calcular_longitud_texto_y_respuesta(df_gemma_sin_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.5")
calcular_longitud_texto_y_respuesta(df_gemma_con_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.1")
calcular_longitud_texto_y_respuesta(df_gemma_sin_rol_0_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.1")
calcular_longitud_texto_y_respuesta(df_gemma_con_rol_0_1)

Longitud textos y respuestas sin rol a 1
Longitud de los textos: 1440.46
Longitud de las respuestas: 95.02
----------------------------------------------------
Longitud textos y respuestas con rol a 1
Longitud de los textos: 1458.46
Longitud de las respuestas: 94.96666666666667
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.5
Longitud de los textos: 1440.46
Longitud de las respuestas: 95.31333333333333
----------------------------------------------------
Longitud textos y respuestas con rol a 0.5
Longitud de los textos: 1458.46
Longitud de las respuestas: 94.72
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.1
Longitud de los textos: 1440.46
Longitud de las respuestas: 95.18666666666667
----------------------------------------------------
Longitud textos y respuestas con rol a 0.1
Longitud de los textos: 1458.46
Longitud de las respuestas: 94.60666666666667


,longitud_textos,longitud_respuestas
0,1458.46,94.606667


# Pruebas llama

## Calidad de generación del texto: BLEU

In [12]:
df_llama_sin_rol_1 = pd.read_csv("./analysis/llama_sin_rol_1.csv")
df_llama_con_rol_1 = pd.read_csv("./analysis/llama_con_rol_1.csv")
df_llama_sin_rol_0_5 = pd.read_csv("./analysis/llama_sin_rol_0_5.csv")
df_llama_con_rol_0_5 = pd.read_csv("./analysis/llama_con_rol_0_5.csv")
df_llama_sin_rol_0_1 = pd.read_csv("./analysis/llama_sin_rol_0_1.csv")
df_llama_con_rol_0_1 = pd.read_csv("./analysis/llama_con_rol_0_1.csv")


print("Llama sin rol 1")
df_bleu_llama_sin_rol_1 = calcular_bleu(df_llama_sin_rol_1)
print("Llama con rol 1")
df_bleu_llama_con_rol_1 = calcular_bleu(df_llama_con_rol_1)
print("Llama sin rol 0.5")
df_bleu_llama_sin_rol_0_5 = calcular_bleu(df_llama_sin_rol_0_5)
print("Llama con rol 0.5")
df_bleu_llama_con_rol_0_5 = calcular_bleu(df_llama_con_rol_0_5)
print("Llama sin rol 0.1")
df_bleu_llama_sin_rol_0_1 = calcular_bleu(df_llama_sin_rol_0_1)
print("Llama con rol 0.1")
df_bleu_llama_con_rol_0_1 = calcular_bleu(df_llama_con_rol_0_1)

Llama sin rol 1
{'bleu': 2.756384901940513e-07, 'precisions': [0.7622970680881996, 0.260678545277032, 0.10508974674207032, 0.061332672776814465], 'brevity_penalty': 1.4570331028759703e-06, 'length_ratio': 0.06925635423274297, 'translation_length': 20635, 'reference_length': 297951}
Llama con rol 1
{'bleu': 3.219687978006532e-07, 'precisions': [0.8143569393569393, 0.43202091664103354, 0.2692705104360405, 0.20207226908257836], 'brevity_penalty': 8.655855826818002e-07, 'length_ratio': 0.06684554720099574, 'translation_length': 19656, 'reference_length': 294051}
Llama sin rol 0.5
{'bleu': 5.7681761830072085e-08, 'precisions': [0.8277514691618646, 0.4618748920987512, 0.3053926975097231, 0.24231422380980266], 'brevity_penalty': 1.4064459583071183e-07, 'length_ratio': 0.059605306562467056, 'translation_length': 17527, 'reference_length': 294051}
Llama con rol 0.5
{'bleu': 3.645467783442035e-07, 'precisions': [0.83871459805766, 0.4749705385048932, 0.3132131977074405, 0.24384659416141957], 'bre

In [16]:
dataframes_llama_bleu = {
    'df_bleu_llama_sin_rol_1': df_bleu_llama_sin_rol_1,
    'df_bleu_llama_con_rol_1': df_bleu_llama_con_rol_1,
    'df_bleu_llama_sin_rol_0_5': df_bleu_llama_sin_rol_0_5,
    'df_bleu_llama_con_rol_0_5': df_bleu_llama_con_rol_0_5,
    'df_bleu_llama_sin_rol_0_1': df_bleu_llama_sin_rol_0_1,
    'df_bleu_llama_con_rol_0_1': df_bleu_llama_con_rol_0_1
}
# Crear gráfico para Llama
grafico_bleu_llama = crear_grafico_bleus(dataframes_llama_bleu, modelo='llama')
# Mostrar gráfico de Llama
output_notebook()
show(grafico_bleu_llama)

Loading BokehJS ...

## Comparación entre pruebas con rol y sin rol a diferentes temperaturas

### Bert por defecto

In [23]:
print ("BertScore por defecto a temperatura 1")
df_bertscore_llama_1 = calcular_bertscore_defecto(df_llama_sin_rol_1, df_llama_con_rol_1)
print ("BertScore por defecto a temperatura 0.5")
df_bertscore_llama_0_5 = calcular_bertscore_defecto(df_llama_sin_rol_0_5, df_llama_con_rol_0_5)
print ("BertScore por defecto a temperatura 0.1")
df_bertscore_llama_0_1 = calcular_bertscore_defecto(df_llama_sin_rol_0_1, df_llama_con_rol_0_1)

BertScore por defecto a temperatura 1
Precision: 0.8054464050134023
Recall: 0.8010353072484334
F1: 0.8030827442804972
BertScore por defecto a temperatura 0.5
Precision: 0.7990198000272115
Recall: 0.8010387722651163
F1: 0.7997263622283936
BertScore por defecto a temperatura 0.1
Precision: 0.7932461086908976
Recall: 0.7972064991792043
F1: 0.794953336318334


In [61]:
dataframes_bert_def_llama = {
    'df_bertscore_llama_1': df_bertscore_llama_1,
    'df_bertscore_llama_0_5': df_bertscore_llama_0_5,
    'df_bertscore_llama_0_1': df_bertscore_llama_0_1
}
# Crear gráfico de BERTScore por defecto para Llama
grafico_bertscore_defecto_llama = crear_grafico_bertscore(dataframes_bert_def_llama, metrica='f1', titulo="BERTScore F1 de Llama por Temperatura")
# Mostrar gráfico de BERTScore por defecto para Llama
output_notebook()
show(grafico_bertscore_defecto_llama)

Loading BokehJS ...

### Bert utilizando modelo Bio_ClinicalBERT

In [36]:
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 1")
df_bertscorebioclinical_llama_1 = calcular_bertscore_bioclinical(df_llama_sin_rol_1, df_llama_con_rol_1)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.5")
df_bertscorebioclinical_llama_0_5 = calcular_bertscore_bioclinical(df_llama_sin_rol_0_5, df_llama_con_rol_0_5)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.1")
df_bertscorebioclinical_llama_0_1 = calcular_bertscore_bioclinical(df_llama_sin_rol_0_1, df_llama_con_rol_0_1)

BertScore con modelo Bio_ClinicalBERT a temperatura 1


Precission: 0.7426342368125916
Recall: 0.7288378477096558
F1: 0.7354907393455505
BertScore con modelo Bio_ClinicalBERT a temperatura 0.5
Precission: 0.7294520139694214
Recall: 0.7297423481941223
F1: 0.7293215394020081
BertScore con modelo Bio_ClinicalBERT a temperatura 0.1
Precission: 0.723676323890686
Recall: 0.7207764983177185
F1: 0.7219652533531189


In [85]:
dataframes_bert_bioclinical_llama = {
    'df_bertscorebioclinical_llama_1': df_bertscorebioclinical_llama_1,
    'df_bertscorebioclinical_llama_0_5': df_bertscorebioclinical_llama_0_5,
    'df_bertscorebioclinical_llama_0_1': df_bertscorebioclinical_llama_0_1
}
# Crear gráfico de BERTScore con Bio_ClinicalBERT para Llama
grafico_bertscore_bioclinical_llama = crear_grafico_bertscore(dataframes_bert_bioclinical_llama, metrica='f1', titulo="BERTScore F1 de Llama con Bio_ClinicalBERT por Temperatura")
# Mostrar gráfico de BERTScore con Bio_ClinicalBERT para Llama
output_notebook()
show(grafico_bertscore_bioclinical_llama)

Loading BokehJS ...

## Medidas de readabilidad y comprensión del texto

In [13]:
print("Metricas de readabilty y comprensión para sin rol a 1")
df_legibilidad_llama_sin_rol_1 = calcular_metricas_comprension(df_llama_sin_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 1")
df_legibilidad_llama_con_rol_1 = calcular_metricas_comprension(df_llama_con_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.5")
df_legibilidad_llama_sin_rol_0_5 = calcular_metricas_comprension(df_llama_sin_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.5")
df_legibilidad_llama_con_rol_0_5 = calcular_metricas_comprension(df_llama_con_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.1")
df_legibilidad_llama_sin_rol_0_1 = calcular_metricas_comprension(df_llama_sin_rol_0_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.1")
df_legibilidad_llama_con_rol_0_1 = calcular_metricas_comprension(df_llama_con_rol_0_1)

Metricas de readabilty y comprensión para sin rol a 1
Flesch Reading Ease Scores: 35.73721710962494
SMOG Index Scores: 13.092083281269657
Dale-Chall Index Scores: 7.97569182076518
Sentiment Polarity Scores: 0.0727435064935065
Sentiment Subjectivity Scores: 0.37562229437229433
Lexical Diversity Scores: 0.7285361261743131
Lexical Richness Scores: 0.015097966041170915
----------------------------------------------------
Metricas de readabilty y comprensión para con rol a 1
Flesch Reading Ease Scores: 39.55900010851506
SMOG Index Scores: 10.179524572218234
Dale-Chall Index Scores: 6.11039722890901
Sentiment Polarity Scores: -0.027253787878787888
Sentiment Subjectivity Scores: 0.19066693722943723
Lexical Diversity Scores: 0.7522334319075863
Lexical Richness Scores: 0.015713019960498238
----------------------------------------------------
Metricas de readabilty y comprensión para sin rol a 0.5
Flesch Reading Ease Scores: 39.230504234887384
SMOG Index Scores: 13.87000152930687
Dale-Chall Inde

## Calcular longitud media de los textos

In [48]:
print("Longitud textos y respuestas sin rol a 1")
calcular_longitud_texto_y_respuesta(df_llama_sin_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 1")
calcular_longitud_texto_y_respuesta(df_llama_con_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.5")
calcular_longitud_texto_y_respuesta(df_llama_sin_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.5")
calcular_longitud_texto_y_respuesta(df_llama_con_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.1")
calcular_longitud_texto_y_respuesta(df_llama_sin_rol_0_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.1")
calcular_longitud_texto_y_respuesta(df_llama_con_rol_0_1)

Longitud textos y respuestas sin rol a 1
Longitud de los textos: 1440.46
Longitud de las respuestas: 95.52
----------------------------------------------------
Longitud textos y respuestas con rol a 1
Longitud de los textos: 1438.46
Longitud de las respuestas: 80.43333333333334
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.5
Longitud de los textos: 1438.46
Longitud de las respuestas: 85.68666666666667
----------------------------------------------------
Longitud textos y respuestas con rol a 0.5
Longitud de los textos: 1438.46
Longitud de las respuestas: 79.17333333333333
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.1
Longitud de los textos: 1438.46
Longitud de las respuestas: 87.4
----------------------------------------------------
Longitud textos y respuestas con rol a 0.1
Longitud de los textos: 1438.46
Longitud de las respuestas: 81.4


,longitud_textos,longitud_respuestas
0,1438.46,81.4


# Pruebas Qwen 

## Calidad de generación del texto: BLEU

In [14]:
df_qwen_sin_rol_1 = pd.read_csv("./analysis/qwen_sin_rol_1.csv")
df_qwen_con_rol_1 = pd.read_csv("./analysis/qwen_con_rol_1.csv")
df_qwen_sin_rol_0_5 = pd.read_csv("./analysis/qwen_sin_rol_0_5.csv")
df_qwen_con_rol_0_5 = pd.read_csv("./analysis/qwen_con_rol_0_5.csv")
df_qwen_sin_rol_0_1 = pd.read_csv("./analysis/qwen_sin_rol_0_1.csv")
df_qwen_con_rol_0_1 = pd.read_csv("./analysis/qwen_con_rol_0_1.csv")


print("Qwen sin rol 1")
df_bleu_qwen_sin_rol_1 = calcular_bleu(df_qwen_sin_rol_1)
print("Qwen con rol 1")
df_bleu_qwen_con_rol_1 = calcular_bleu(df_qwen_con_rol_1)
print("Qwen sin rol 0.5")
df_bleu_qwen_sin_rol_0_5 =calcular_bleu(df_qwen_sin_rol_0_5)
print("Qwen con rol 0.5")
df_bleu_qwen_con_rol_0_5 =calcular_bleu(df_qwen_con_rol_0_5)
print("Qwen sin rol 0.1")
df_bleu_qwen_sin_rol_0_1 =calcular_bleu(df_qwen_sin_rol_0_1)
print("Qwen con rol 0.1")
df_bleu_qwen_con_rol_0_1 = calcular_bleu(df_qwen_con_rol_0_1)


Qwen sin rol 1
{'bleu': 0.05103363921814337, 'precisions': [0.4685189859723775, 0.15742077528333212, 0.06445455968594584, 0.030710635425706297], 'brevity_penalty': 0.46427320179280335, 'length_ratio': 0.5658406194843751, 'translation_length': 166386, 'reference_length': 294051}
Qwen con rol 1
{'bleu': 0.04946913983663019, 'precisions': [0.47420587312427326, 0.1599204369807928, 0.06511461627125747, 0.030859970262883514], 'brevity_penalty': 0.44524406353237217, 'length_ratio': 0.5527510533886979, 'translation_length': 162537, 'reference_length': 294051}
Qwen sin rol 0.5
{'bleu': 0.05254070432765501, 'precisions': [0.4682774225517764, 0.15991729857961506, 0.06606498842696427, 0.03163143748241534], 'brevity_penalty': 0.46975742288407113, 'length_ratio': 0.5696256771784486, 'translation_length': 167499, 'reference_length': 294051}
Qwen con rol 0.5
{'bleu': 0.05371851471876594, 'precisions': [0.47842532584865205, 0.16575434958222307, 0.06887188646456845, 0.03331964391404655], 'brevity_penalt

In [18]:
dataframes_bleu_qwen = {
    'df_bleu_qwen_sin_rol_1': df_bleu_qwen_sin_rol_1,
    'df_bleu_qwen_con_rol_1': df_bleu_qwen_con_rol_1,
    'df_bleu_qwen_sin_rol_0_5': df_bleu_qwen_sin_rol_0_5,
    'df_bleu_qwen_con_rol_0_5': df_bleu_qwen_con_rol_0_5,
    'df_bleu_qwen_sin_rol_0_1': df_bleu_qwen_sin_rol_0_1,
    'df_bleu_qwen_con_rol_0_1': df_bleu_qwen_con_rol_0_1
}
# Crear gráfico para Qwen
grafico_bleu_qwen = crear_grafico_bleus(dataframes_bleu_qwen, modelo='qwen')
# Mostrar gráfico de Qwen
output_notebook()
show(grafico_bleu_qwen)

Loading BokehJS ...

## Comparación entre pruebas con rol y sin rol a diferentes temperaturas

### Bert por defecto

In [24]:
print ("BertScore por defecto a temperatura 1")
df_bertscore_qwen_1 = calcular_bertscore_defecto(df_qwen_sin_rol_1, df_qwen_con_rol_1)
print ("BertScore por defecto a temperatura 0.5")
df_bertscore_qwen_0_5 = calcular_bertscore_defecto(df_qwen_sin_rol_0_5, df_qwen_con_rol_0_5)
print ("BertScore por defecto a temperatura 0.1")
df_bertscore_qwen_0_1 = calcular_bertscore_defecto(df_qwen_sin_rol_0_1, df_qwen_con_rol_0_1)

BertScore por defecto a temperatura 1
Precision: 0.9003736313184102
Recall: 0.9004648605982463
F1: 0.9003602476914724
BertScore por defecto a temperatura 0.5
Precision: 0.9024180607000987
Recall: 0.8999850408236185
F1: 0.9011495343844096
BertScore por defecto a temperatura 0.1
Precision: 0.9025327304999033
Recall: 0.9036619325478872
F1: 0.9030515038967133


### Bert utilizando modelo Bio_ClinicalBERT

In [37]:
def calcular_bertscore_bioclinical(df_sin_rol, df_con_rol):
    
    def truncate(text, max_len=1000):
        return str(text)[:max_len]

    predictions = df_sin_rol["output"].astype(str).apply(truncate).tolist()
    references = df_con_rol["output"].astype(str).apply(truncate).tolist()


    P, R, F1 = score(predictions, references, lang="en", num_layers=12, idf=False, model_type="emilyalsentzer/Bio_ClinicalBERT")
    print(f"Precission: {sum(P) / len(P)}")
    print(f"Recall: {sum(R) / len(R)}")
    print(f"F1: {sum(F1) / len(F1)}")

    df = pd.DataFrame({
        "precision": [(sum(P) / len(P)).item()],
        "recall": [(sum(R) / len(R)).item()],
        "f1": [(sum(F1) / len(F1)).item()]
    })
    return df

    

In [51]:
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 1")
df_bertscorebioclinical_qwen_1 = calcular_bertscore_bioclinical(df_qwen_sin_rol_1, df_qwen_con_rol_1)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.5")
df_bertscorebioclinical_qwen_0_5 = calcular_bertscore_bioclinical(df_qwen_sin_rol_0_5, df_qwen_con_rol_0_5)
print ("BertScore con modelo Bio_ClinicalBERT a temperatura 0.1")
df_bertscorebioclinical_qwen_0_1 = calcular_bertscore_bioclinical(df_qwen_sin_rol_0_1, df_qwen_con_rol_0_1)

BertScore con modelo Bio_ClinicalBERT a temperatura 1
Precission: 0.8725554347038269
Recall: 0.8720989227294922
F1: 0.8722508549690247
BertScore con modelo Bio_ClinicalBERT a temperatura 0.5
Precission: 0.874184787273407
Recall: 0.8741495013237
F1: 0.8741008639335632
BertScore con modelo Bio_ClinicalBERT a temperatura 0.1
Precission: 0.8722729682922363
Recall: 0.8716810941696167
F1: 0.8718870282173157


## Medidas de readabilidad y comprensión del texto

In [15]:
print("Metricas de readabilty y comprensión para sin rol a 1")
df_legibilidad_qwen_sin_rol_1 = calcular_metricas_comprension(df_qwen_sin_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 1")
df_legibilidad_qwen_con_rol_1 = calcular_metricas_comprension(df_qwen_con_rol_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.5")
df_legibilidad_qwen_sin_rol_0_5 = calcular_metricas_comprension(df_qwen_sin_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.5")
df_legibilidad_qwen_con_rol_0_5 = calcular_metricas_comprension(df_qwen_con_rol_0_5)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para sin rol a 0.1")
df_legibilidad_qwen_sin_rol_0_1 = calcular_metricas_comprension(df_qwen_sin_rol_0_1)
print("----------------------------------------------------")
print("Metricas de readabilty y comprensión para con rol a 0.1")
df_legibilidad_qwen_con_rol_0_1 = calcular_metricas_comprension(df_qwen_con_rol_0_1)

Metricas de readabilty y comprensión para sin rol a 1
Flesch Reading Ease Scores: 44.219149946574234
SMOG Index Scores: 9.776128628462624
Dale-Chall Index Scores: 9.25085023117661
Sentiment Polarity Scores: 0.053972354894246385
Sentiment Subjectivity Scores: 0.3556468644234862
Lexical Diversity Scores: 0.6418819584734294
Lexical Richness Scores: 0.002730976856671808
----------------------------------------------------
Metricas de readabilty y comprensión para con rol a 1
Flesch Reading Ease Scores: 43.96120919893773
SMOG Index Scores: 9.508993488578366
Dale-Chall Index Scores: 9.31399928002116
Sentiment Polarity Scores: 0.07415897457502899
Sentiment Subjectivity Scores: 0.39310308459459364
Lexical Diversity Scores: 0.6645496352426247
Lexical Richness Scores: 0.0028832806933534196
----------------------------------------------------
Metricas de readabilty y comprensión para sin rol a 0.5
Flesch Reading Ease Scores: 38.115430449933
SMOG Index Scores: 10.890181073343683
Dale-Chall Index S

## Calcular longitud media de los textos

In [23]:
print("Longitud textos y respuestas sin rol a 1")
calcular_longitud_texto_y_respuesta(df_qwen_sin_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 1")
calcular_longitud_texto_y_respuesta(df_qwen_con_rol_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.5")
calcular_longitud_texto_y_respuesta(df_qwen_sin_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.5")
calcular_longitud_texto_y_respuesta(df_qwen_con_rol_0_5)
print("----------------------------------------------------")
print("Longitud textos y respuestas sin rol a 0.1")
calcular_longitud_texto_y_respuesta(df_qwen_sin_rol_0_1)
print("----------------------------------------------------")
print("Longitud textos y respuestas con rol a 0.1")
calcular_longitud_texto_y_respuesta(df_qwen_con_rol_0_1)

Longitud textos y respuestas sin rol a 1
Longitud de los textos: 1438.46
Longitud de las respuestas: 576.4066666666666
----------------------------------------------------
Longitud textos y respuestas con rol a 1
Longitud de los textos: 1438.46
Longitud de las respuestas: 566.7
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.5
Longitud de los textos: 1438.46
Longitud de las respuestas: 581.32
----------------------------------------------------
Longitud textos y respuestas con rol a 0.5
Longitud de los textos: 1438.46
Longitud de las respuestas: 579.56
----------------------------------------------------
Longitud textos y respuestas sin rol a 0.1
Longitud de los textos: 1438.46
Longitud de las respuestas: 578.3933333333333
----------------------------------------------------
Longitud textos y respuestas con rol a 0.1
Longitud de los textos: 1438.46
Longitud de las respuestas: 586.6933333333334


,longitud_textos,longitud_respuestas
0,1438.46,586.693333


# Graficas todos los modelos

## Gráficas bert por defecto

In [86]:
dataframes_bert = {
    'df_bertscore_gemma_1': df_bertscore_gemma_1,
    'df_bertscore_gemma_0_5': df_bertscore_gemma_0_5,
    'df_bertscore_gemma_0_1': df_bertscore_gemma_0_1,
    'df_bertscore_llama_1': df_bertscore_llama_1,
    'df_bertscore_llama_0_5': df_bertscore_llama_0_5,
    'df_bertscore_llama_0_1': df_bertscore_llama_0_1,
    'df_bertscore_qwen_1': df_bertscore_qwen_1,
    'df_bertscore_qwen_0_5': df_bertscore_qwen_0_5,
    'df_bertscore_qwen_0_1': df_bertscore_qwen_0_1
}
# Crear gráfico de BERTScor
grafico_bertscore = crear_grafico_bertscore(dataframes_bert, metrica='f1', titulo="BERTScore F1 de todos los modelos con BERT por defecto por Temperatura")
# Mostrar gráfico
output_notebook()
show(grafico_bertscore)

Loading BokehJS ...

In [87]:
# Crear gráfico de BERTScor
grafico_bertscore = crear_grafico_bertscore(dataframes_bert, metrica='precision', titulo="BERTScore Precision de todos los modelos con BERT por defecto por Temperatura")
# Mostrar gráfico
output_notebook()
show(grafico_bertscore)

Loading BokehJS ...

In [89]:
# Crear gráfico de BERTScor
grafico_bertscore = crear_grafico_bertscore(dataframes_bert, metrica='recall', titulo="BERTScore recall de todos los modelos con BERT por defecto por Temperatura")
# Mostrar gráfico
output_notebook()
show(grafico_bertscore)

Loading BokehJS ...

## Gráficas bert bioClinical_Bert

In [90]:
dataframes_bert = {
    'df_bertscorebioclinical_gemma_1': df_bertscorebioclinical_gemma_1,
    'df_bertscorebioclinical_gemma_0_5': df_bertscorebioclinical_gemma_0_5,
    'df_bertscorebioclinical_gemma_0_1': df_bertscorebioclinical_gemma_0_1,
    'df_bertscorebioclinical_llama_1': df_bertscorebioclinical_llama_1,
    'df_bertscorebioclinical_llama_0_5': df_bertscorebioclinical_llama_0_5,
    'df_bertscorebioclinical_llama_0_1': df_bertscorebioclinical_llama_0_1,
    'df_bertscorebioclinical_qwen_1': df_bertscorebioclinical_qwen_1,
    'df_bertscorebioclinical_qwen_0_5': df_bertscorebioclinical_qwen_0_5,
    'df_bertscorebioclinical_qwen_0_1': df_bertscorebioclinical_qwen_0_1
}
# Crear gráfico de BERTScor
grafico_bertscorebioclinical = crear_grafico_bertscore(dataframes_bert, metrica='f1', titulo="BERTScore F1 de todos los modelos con BioClinical_BERT por Temperatura")
# Mostrar gráfico
output_notebook()
show(grafico_bertscorebioclinical)

Loading BokehJS ...

In [91]:
# Crear gráfico de BERTScor
grafico_bertscorebioclinical = crear_grafico_bertscore(dataframes_bert, metrica='precision', titulo="BERTScore precision de todos los modelos con BioClinical_BERT por Temperatura")
# Mostrar gráfico
output_notebook()
show(grafico_bertscorebioclinical)

Loading BokehJS ...

In [92]:
# Crear gráfico de BERTScor
grafico_bertscorebioclinical = crear_grafico_bertscore(dataframes_bert, metrica='recall', titulo="BERTScore recall de todos los modelos con BioClinical_BERT por Temperatura")
# Mostrar gráfico
output_notebook()
show(grafico_bertscorebioclinical)

Loading BokehJS ...

## Gráficas readability y comprensión del texto

In [16]:
dataframes_legibilidad = {
    'df_legibilidad_gemma_sin_rol_1': df_legibilidad_gemma_sin_rol_1,
    'df_legibilidad_gemma_con_rol_1': df_legibilidad_gemma_con_rol_1,
    'df_legibilidad_gemma_sin_rol_0_5': df_legibilidad_gemma_sin_rol_0_5,
    'df_legibilidad_gemma_con_rol_0_5': df_legibilidad_gemma_con_rol_0_5,
    'df_legibilidad_gemma_sin_rol_0_1': df_legibilidad_gemma_sin_rol_0_1,
    'df_legibilidad_gemma_con_rol_0_1': df_legibilidad_gemma_con_rol_0_1,
    'df_legibilidad_llama_sin_rol_1': df_legibilidad_llama_sin_rol_1,
    'df_legibilidad_llama_con_rol_1': df_legibilidad_llama_con_rol_1,
    'df_legibilidad_llama_sin_rol_0_5': df_legibilidad_llama_sin_rol_0_5,
    'df_legibilidad_llama_con_rol_0_5': df_legibilidad_llama_con_rol_0_5,
    'df_legibilidad_llama_sin_rol_0_1': df_legibilidad_llama_sin_rol_0_1,
    'df_legibilidad_llama_con_rol_0_1': df_legibilidad_llama_con_rol_0_1,
    'df_legibilidad_qwen_sin_rol_1': df_legibilidad_qwen_sin_rol_1,
    'df_legibilidad_qwen_con_rol_1': df_legibilidad_qwen_con_rol_1,
    'df_legibilidad_qwen_sin_rol_0_5': df_legibilidad_qwen_sin_rol_0_5,
    'df_legibilidad_qwen_con_rol_0_5': df_legibilidad_qwen_con_rol_0_5,
    'df_legibilidad_qwen_sin_rol_0_1': df_legibilidad_qwen_sin_rol_0_1,
    'df_legibilidad_qwen_con_rol_0_1': df_legibilidad_qwen_con_rol_0_1
}
# Crear gráfico de legibilidad
grafico_legibilidad = crear_grafico_legibilidad(dataframes_legibilidad, metrica='flesch_reading_ease', titulo="Flesch Reading Ease de todos los modelos por Temperatura y Rol")
# Mostrar gráfico
output_notebook()
show(grafico_legibilidad)

Loading BokehJS ...

In [17]:
# Crear gráfico de legibilidad
grafico_legibilidad = crear_grafico_legibilidad(dataframes_legibilidad, metrica='smog_index', titulo="Smog Index de todos los modelos por Temperatura y Rol")
# Mostrar gráfico
output_notebook()
show(grafico_legibilidad)

Loading BokehJS ...

In [18]:
# Crear gráfico de legibilidad
grafico_legibilidad = crear_grafico_legibilidad(dataframes_legibilidad, metrica='dale_chall_index', titulo="Dale Chall Index de todos los modelos por Temperatura y Rol")
# Mostrar gráfico
output_notebook()
show(grafico_legibilidad)

Loading BokehJS ...

In [24]:
# Crear gráfico de legibilidad
grafico_legibilidad = crear_grafico_legibilidad(dataframes_legibilidad, metrica='sentiment_polarity', titulo="Puntuación de polaridad de sentimientos de todos los modelos por Temperatura y Rol")
# Mostrar gráfico
output_notebook()
show(grafico_legibilidad)

Loading BokehJS ...

In [20]:
# Crear gráfico de legibilidad
grafico_legibilidad = crear_grafico_legibilidad(dataframes_legibilidad, metrica='sentiment_subjectivity', titulo="Puntuación de subjetividad de sentimientos de todos los modelos por Temperatura y Rol")
# Mostrar gráfico
output_notebook()
show(grafico_legibilidad)

Loading BokehJS ...

In [21]:
# Crear gráfico de legibilidad
grafico_legibilidad = crear_grafico_legibilidad(dataframes_legibilidad, metrica='lexical_diversity', titulo="Puntuación de diversidad léxica de todos los modelos por Temperatura y Rol")
# Mostrar gráfico
output_notebook()
show(grafico_legibilidad)

Loading BokehJS ...

In [22]:
# Crear gráfico de legibilidad
grafico_legibilidad = crear_grafico_legibilidad(dataframes_legibilidad, metrica='lexical_richness', titulo="Puntuación de riqueza léxica de todos los modelos por Temperatura y Rol")
# Mostrar gráfico
output_notebook()
show(grafico_legibilidad)

Loading BokehJS ...